In [435]:
class Node:
    def __init__(self, x, y):
        self.wage = 0
        self.wage_out = 0
        self.wage_in = 0
        self.coordinates = (x,y)
        self.edges_in = []
        self.edges_out = []

class Edge:
    def __init__(self, node1, node2):
        self.node1 = node1
        self.node2 = node2
        self.wage = 1

class Graph:
    def __init__(self):
        self.nodes = []
        self.edges = []

### Wyznaczanie wag krawedzi

In [436]:
def sort_from_left(v, edges):

    def a(a,b):
        return (a[1]-b[1])/(a[0]-b[0])
    
    def cmp(e):
        vcoordinates = v.coordinates
        if e.node2 != v:
            orient = a(vcoordinates, e.node2.coordinates)
            i = 1
        else:
            orient = a(vcoordinates, e.node1.coordinates)
            i = -1
        if orient < 0:
            return(-1*i,-orient*i)
        elif orient > 0:
            return(1*i,-orient*i)
        else:
            return(0)
    
    if len(edges) == 0:
        return None
    edges.sort(key = lambda e: cmp(e))
    return edges

In [437]:
def edgeswages(graph):
    nodes = graph.nodes
    nodes.sort(key=lambda x: (x.coordinates[1],x.coordinates[0]))
    for v in nodes:
        v.wage_in = 0
        for v_in in v.edges_in:
            v.wage_in += v_in.wage
        edges_sorted = sort_from_left(v, v.edges_out)
        if edges_sorted != None and  v.wage_in > len(v.edges_out):
            d = edges_sorted[0]
            d.wage += v.wage_in - len(v.edges_out)
    nodes.reverse()
    for v in nodes:
        v.wage_out = 0
        for v_out in v.edges_out:
            v.wage_out += v_out.wage
        edges_sorted = sort_from_left(v, v.edges_in)
        if edges_sorted != None and  v.wage_out > v.wage_in:
            d = edges_sorted[0]
            d.wage += v.wage_out - v.wage_in
    return

## Przykład do testów

In [438]:
n1 = Node(10,0)
n2 = Node(12,2)
n3 = Node(5,5)
n4 = Node(11,7)
n5 = Node(15,8)
n6 = Node(10,9)
n7 = Node(14,10)
n8 = Node(0,12)
n9 = Node(9,15)
nodes = [n1, n2, n3, n4, n5, n6, n7, n8, n9]

e1 = Edge(n1,n2)
e2 = Edge(n1,n3)
e3 = Edge(n1,n4)
e4 = Edge(n1,n5)
e5 = Edge(n2,n5)
e6 = Edge(n3,n4)
e7 = Edge(n3,n6)
e8 = Edge(n3,n8)
e9 = Edge(n4,n5)
e10 = Edge(n4,n6)
e11 = Edge(n5,n6)
e12 = Edge(n5,n7)
e13 = Edge(n6,n7)
e14 = Edge(n6,n9)
e15 = Edge(n7,n9)
e16 = Edge(n8,n9)
edges = [e1, e2, e3, e4, e5, e6, e7, e8, e9, e10, e11, e12, e13, e14, e15, e16]

for e in edges:
    e.node1.edges_out.append(e)
    e.node2.edges_in.append(e)

graph = Graph()
graph.edges = edges
graph.nodes = nodes

In [439]:
# edgeswages(graph)
# sorted = sort_from_left(n1, n1.edges_out)
# for edge in sorted:
#     print(edge.node2.coordinates)

In [440]:
# for e in edges:
#     print(f"waga: {e.wage}, wierz: {e.node1.coordinates}, {e.node2.coordinates}")

### Wyznaczanie separatorów

In [441]:
def find_separators(graph):
    edgeswages(graph)
    nodes = graph.nodes
    nodes.sort(key=lambda x: (x.coordinates[1],x.coordinates[0]))
    separators = []

    def find(v, sep, v_k):
        if v == v_k:
            return sep
        if v.edges_out == None:
            return sep
        v.edges_out = sort_from_left(v,v.edges_out)
        i = 0
        while len(v.edges_out) > i and v.edges_out[i].wage == 0:
            i += 1
        if len(v.edges_out) <= i:
            return None
        v.edges_out[i].wage -=1
        sep.append(v.edges_out[i])
        return(find(v.edges_out[i].node2, sep, v_k))

    for v in range(0,len(nodes)-1):
        nodes[v].edges_out = sort_from_left(nodes[v], nodes[v].edges_out)
    separator = find(nodes[0],[], nodes[-1])
    while separator != None:

        separators.append(separator)
        separator = find(nodes[0],[], nodes[-1])
    return separators


In [442]:
separators = find_separators(graph)
for s in separators:
    for v in s:
        print(v.node2.coordinates)
    print("\n")
print(len(separators))


(5, 5)
(0, 12)
(9, 15)


(5, 5)
(10, 9)
(9, 15)


(5, 5)
(11, 7)
(10, 9)
(9, 15)


(11, 7)
(15, 8)
(10, 9)
(9, 15)


(15, 8)
(10, 9)
(14, 10)
(9, 15)


(12, 2)
(15, 8)
(14, 10)
(9, 15)


6


### Drzewo Binarne przechowujące separatory

In [443]:
class TreeNode:
    def __init__(self, edges, id, parent):
        self.id = id
        self.parent = parent
        self.right = None
        self.left = None
        self.edges=edges

In [444]:
def make_Tree(separators):

    # Zamień used na hashmape!!
    # Optymalizacja by nie pamietac wszystkich krawedzi

    def delete_used(separatorEdges, used):
        i=0
        while i < len(separatorEdges):
            if separatorEdges[i] in used:
                separatorEdges.pop(i)
            else:
                used.append(separatorEdges[i])
                i += 1
        return separatorEdges, used
    
    def create(start, stop, used, parent):
        if stop-start == 0:
            return None
        sr = (stop+start)//2
        print(sr)
        new_edges, used = delete_used(separators[sr], used)
        new = TreeNode(new_edges, sr, parent)
        new.left = create(start, sr, used, new)
        new.right = create(sr+1,stop, used, new)
        return new
    
    print(len(separators))
    top = create(0, len(separators), [], None)
    return top


In [445]:
top = make_Tree(separators)

6
3
1
0
2
5
4


In [446]:
def print_Tree(top):
    print(top.id)
    if top.left != None:
        print_Tree(top.left)
    if top.right != None:
        print_Tree(top.right)
    return

In [447]:
print_Tree(top)

3
1
0
2
5
4


### Lokalizacja punktu

In [448]:
def Orientation(a, b, c):
    temp1 = a[0] * b[1] - c[0] * b[1]
    temp2 = b[0] * c[1] - c[1] * a[0]
    temp3 = a[1] * c[0] - a[1] * b[0]
    return temp1 + temp2 + temp3


def right_left(point, edge):
    orient = Orientation(edge.node1.coordinates, edge.node2.coordinates, point)
    return orient


In [449]:
def localize_point(point, top, last):
    separator = top.edges
    i = 0
    while len(separator) > i and separator[i].node2.coordinates[1] < point[1]:
        i += 1
    if i >= len(separator):
        orient = last
    else:
        orient = right_left(point, separator[i])
    if  orient < 0:
        if top.right == None:
            return (top.id + 1, top.id+2)
        else:
            return localize_point(point, top.right, -1)
    else:
        if top.left == None:
            return (top.id, top.id+1)
        else:
            return localize_point(point, top.left, 1)
    

In [450]:
p = [12,11]
location = localize_point(p,top, None)
print(location)

(4, 5)


In [451]:
### Krawedzie otaczające obszar na którym znajduje się punkt
edges_upper = separators[location[0]-1]
edges_lower = separators[location[1]-1]